# MTH 651: Advanced Numerical Analysis

## Homework Assignment 4

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_651_assignment_4.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is:
    1. To assemble the mass matrix and study its relation to $L^2$ projection.
    2. To study the use of stationary iterative methods for the solution of linear systems arising from finite element discretizations.

The follow code implements the functions `area`, `make_stiffness`, `make_rhs`, and `square_mesh` from the previous assignment.
These will be used in the subsequent questions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def area(K):
    """
    Returns the area of the triangle defined by K.
    """
    M = np.array([[K[0,0], K[0,1], 1],
                  [K[1,0], K[1,1], 1],
                  [K[2,0], K[2,1], 1]])
    return 0.5 * np.linalg.det(M)

def make_stiffness(T, V, B):
    """
    Assembles the stiffness matrix on the mesh defined by (T, V). Eliminates the
    essential boundary conditions defined by B.
    """
    N = V.shape[0]
    A = np.zeros((N, N))

    for it in range(T.shape[0]):
        K = V[T[it,:],:]
        G1 = np.array([[1, 1, 1],
                       [K[0,0], K[1,0], K[2,0]],
                       [K[0,1], K[1,1], K[2,1]]])
        G2 = np.array([[0,0],[1,0],[0,1]])
        G = np.linalg.solve(G1, G2)
        A_K = area(K) * G @ G.T

        A[np.ix_(T[it,:], T[it,:])] += A_K

    A[B,:] = 0.0
    A[:,B] = 0.0
    for i in B:
        A[i,i] = 1.0

    return A

def make_rhs(f, T, V):
    N = V.shape[0]
    F = np.zeros(N)

    for it in range(T.shape[0]):
        K = V[T[it,:], :]
        for iv in range(3):
            x, y = K[iv,:]
            F[T[it,iv]] += 1/3 * area(K) * f(x, y)

    return F

def square_mesh(nx):
    """
    Generates a triangular Cartesian mesh of the unit square with nx vertices in
    each dimension.

    Returns (V, T, B), where V is are the vertex coordinates, T are the triangle
    indices, and B is a list of boundary vertex indices.
    """
    x = np.linspace(0, 1, nx)
    X, Y = np.meshgrid(x, x)
    V = np.stack((X.ravel(), Y.ravel()), axis=1)

    nt = 2*(nx-1)**2
    T = np.zeros((nt, 3), int)

    for iy in range(nx - 1):
        for ix in range(nx - 1):
            v1 = ix + iy*nx
            v2 = ix + 1 + iy*nx
            v3 = ix + (iy + 1)*nx
            v4 = ix + 1 + (iy + 1)*nx
            T[2*ix + iy*2*(nx-1), :] = [v1, v2, v4]
            T[2*ix + 1 + iy*2*(nx-1), :] = [v1, v4, v3]

    B = []
    for i in range(nx):
        B.append(i)
        B.append(i + nx*(nx - 1))
    for i in range(1, nx - 1):
        B.append(nx*i)
        B.append(nx - 1 + nx*i)

    return V, T, B

### Problem 1. (3 points). Mass matrix

Write a function that assembles the mass matrix defined by
$$
    M_{ij} = \int_\Omega \phi_i \phi_j~dx.
$$
No boundary conditions are required.
The mass matrix can be assembled by adding contributions from the **local mass matrices** $M_K$, where $M_K$ is given by
$$
    M_K = \frac{|K|}{12} \begin{pmatrix}
        2 & 1 & 1 \\
        1 & 2 & 1 \\
        1 & 1 & 2 \\
    \end{pmatrix}.
$$
You can use the same technique as in `make_stiffness`.

In [ ]:
def make_mass(T, V):
    """
    Assembles the mass matrix on the mesh defined by (T, V).
    """
    N = V.shape[0]
    M = np.zeros((N, N))
    # Assemble M using the local mass matrices M_K
    return M

### Problem 2. (3 points). Stationary method

Write a function `stationary` that implements a general stationary iterative method to solve
$$
    A x = b
$$
using the matrix $B$ in the iteration
$$
    x^{(i+1)} = x^{(i)} + B ( b - A x^{(i)}).
$$
Terminate the iteration when the $\ell^2$ norm of the residual is below the relative tolerance of $10^{-5}$ (up to a maximum of 200 iterations).
Note, by relative tolerance, we mean that the iteration should terminate when
$$
    \frac{\| r \|_{\ell^2}}{\| b \|_{\ell^2}} \leq \mathrm{tol}.
$$
Record the number of iterations required to achieve convergence.

In [ ]:
def stationary(A, B, b):
    n = len(b)
    x = np.zeros(n)
    # Solve the system with the stationary method
    return x

### Problem 3. (3 points). Richardson method

Create a square mesh with $n_x = 2^i$ for $i \in \{2, 3, 4, 5 \}$.
Assemble the mass and stiffness matrices.
Solve the linear system
$$
    Ax = b
$$
and
$$
    Mx = b
$$
where $b$ is defined by `make_rhs(f, T, V)`, with `f` defined as below.

Solve each system using Richardson's method.
Choose the scaling factor $\omega$ according to the formula for the optimal $\omega$, using the estimates for the minimum and maximum eigenvalues in terms of the mesh size $h$.
(Don't compute the eigenvalues from the matrix, use the analytical estimates — this may require some trial and error).

Do the results agree with the theoretical estimates?

In [ ]:
def f(x, y):
    return x * y

### Problem 4. (3 points). Steepest descent

Repeat the previous problem, this time using the steepest descent method.
How does the convergence compare to that of the Richardson method?

### Problem 5. (3 points). Gauss-Seidel

Repeat the previous problem, this time using the Gauss-Seidel method.

Estimate the convergence rate of Gauss-Seidel applied to $A$ and $M$ in terms of the mesh size $h$ based on the number of iterations required to attain convergence.

### Textbook Problems (3 points each)

* 6.1
* 7.5